In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



## Import required Libraries





In [3]:
import torch
import torchvision
import pathlib
import glob
import os
import numpy as np
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision.transforms as T
import torch.nn as nn
from torchvision import models
from torchvision.models import squeezenet1_1
from PIL import Image
import numpy as np
from torch.optim import Adam


## checking for device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
print(device)

cuda


## transforms

In [6]:
transformer = T.Compose([
    T.Resize((150,150)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.5,0.5,0.5],
                [0.5,0.5,0.5])
])

## Data Loader

In [7]:
train_path = '/content/traffic_sign_classification_dataset/train'
test_path = '/content/traffic_sign_classification_dataset/test'

train_loader = DataLoader(torchvision.datasets.ImageFolder(train_path,transform = transformer),
                          batch_size = 64,shuffle = True)
test_loader = DataLoader(torchvision.datasets.ImageFolder(test_path,transform = transformer),
                          batch_size = 32,shuffle = True)



In [8]:
#categories
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [9]:
print(classes)

['ALL_MOTOR_VEHICLE_PROHIBITED', 'AXLE_LOAD_LIMIT', 'BARRIER_AHEAD', 'BULLOCK_AND_HANDCART_PROHIBITED', 'BULLOCK_PROHIBITED', 'CATTLE', 'COMPULSARY_AHEAD', 'COMPULSARY_AHEAD_OR_TURN_LEFT', 'COMPULSARY_AHEAD_OR_TURN_RIGHT', 'COMPULSARY_CYCLE_TRACK', 'COMPULSARY_KEEP_LEFT', 'COMPULSARY_KEEP_RIGHT', 'COMPULSARY_MINIMUM_SPEED', 'COMPULSARY_SOUND_HORN', 'COMPULSARY_TURN_LEFT', 'COMPULSARY_TURN_LEFT_AHEAD', 'COMPULSARY_TURN_RIGHT', 'COMPULSARY_TURN_RIGHT_AHEAD', 'CROSS_ROAD', 'CYCLE_CROSSING', 'CYCLE_PROHIBITED', 'DANGEROUS_DIP', 'DIRECTION', 'FALLING_ROCKS', 'FERRY', 'GAP_IN_MEDIAN', 'GIVE_WAY', 'GUARDED_LEVEL_CROSSING', 'HANDCART_PROHIBITED', 'HEIGHT_LIMIT', 'HORN_PROHIBITED', 'HUMP_OR_ROUGH_ROAD', 'LEFT_HAIR_PIN_BEND', 'LEFT_HAND_CURVE', 'LEFT_REVERSE_BEND', 'LEFT_TURN_PROHIBITED', 'LENGTH_LIMIT', 'LOAD_LIMIT', 'LOOSE_GRAVEL', 'MEN_AT_WORK', 'NARROW_BRIDGE', 'NARROW_ROAD_AHEAD', 'NO_ENTRY', 'NO_PARKING', 'NO_STOPPING_OR_STANDING', 'OVERTAKING_PROHIBITED', 'PASS_EITHER_SIDE', 'PEDESTRIAN_C

In [10]:
len(classes)

85

In [11]:
#CNN Network

class ConvNet(nn.Module):
    def __init__(self,num_classes=85):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [12]:
model = ConvNet(num_classes = 85).to(device)

In [14]:
optimizer = Adam(model.parameters(),lr=0.001,weight_decay = 0.0001)
loss_function = nn.CrossEntropyLoss()

In [15]:
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))

In [16]:
print(train_count,test_count)

4438 1288


In [17]:
num_epochs = 10

In [18]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
          images=Variable(images.cuda())
          labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'Best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(37.4431) Train Accuracy: 0.25101397025687244 Test Accuracy: 0.3385093167701863
Epoch: 1 Train Loss: tensor(5.9828) Train Accuracy: 0.5164488508337088 Test Accuracy: 0.4518633540372671
Epoch: 2 Train Loss: tensor(3.4219) Train Accuracy: 0.6399278954484002 Test Accuracy: 0.5861801242236024
Epoch: 3 Train Loss: tensor(1.6961) Train Accuracy: 0.7528165840468679 Test Accuracy: 0.6451863354037267
Epoch: 4 Train Loss: tensor(1.0421) Train Accuracy: 0.8217665615141956 Test Accuracy: 0.65527950310559
Epoch: 5 Train Loss: tensor(0.5415) Train Accuracy: 0.885984677782785 Test Accuracy: 0.7104037267080745
Epoch: 6 Train Loss: tensor(0.4517) Train Accuracy: 0.905588102748986 Test Accuracy: 0.7096273291925466
Epoch: 7 Train Loss: tensor(0.4873) Train Accuracy: 0.9062640829202343 Test Accuracy: 0.7406832298136646
Epoch: 8 Train Loss: tensor(0.3058) Train Accuracy: 0.9380351509689049 Test Accuracy: 0.7546583850931677
Epoch: 9 Train Loss: tensor(0.3091) Train Accuracy: 0.948

In [20]:
checkpoint=torch.load('Best_checkpoint.model')
model=ConvNet(num_classes=85)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=85, bias=True)
)

In [21]:
#prediction function
def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [22]:
pred_path = '/content/drive/MyDrive/Traffic signs/pred'
images_path=glob.glob(pred_path+'/*.jpg')

In [23]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [26]:
pred_dict

{'08010.jpg': 'COMPULSARY_AHEAD_OR_TURN_RIGHT',
 '11118.jpg': 'COMPULSARY_KEEP_RIGHT',
 '10011.jpg': 'COMPULSARY_KEEP_LEFT',
 '10074.jpg': 'COMPULSARY_KEEP_LEFT',
 '08011.jpg': 'COMPULSARY_AHEAD_OR_TURN_RIGHT',
 '11119.jpg': 'COMPULSARY_KEEP_RIGHT',
 '08009.jpg': 'COMPULSARY_AHEAD_OR_TURN_RIGHT',
 '10073.jpg': 'CROSS_ROAD',
 '19011.jpg': 'CYCLE_CROSSING',
 '20004.jpg': 'CYCLE_PROHIBITED',
 '23018.jpg': 'SCHOOL_AHEAD',
 '23017.jpg': 'FALLING_ROCKS',
 '23019.jpg': 'SPEED_LIMIT_30',
 '20006.jpg': 'NO_STOPPING_OR_STANDING',
 '20005.jpg': 'CYCLE_PROHIBITED',
 '78012.jpg': 'HUMP_OR_ROUGH_ROAD',
 '63044.jpg': 'SPEED_LIMIT_15',
 '78004.jpg': 'TRAFFIC_SIGNAL',
 '67084.jpg': 'SPEED_LIMIT_60',
 '78008.jpg': 'TRAFFIC_SIGNAL',
 '00015.jpg': 'ALL_MOTOR_VEHICLE_PROHIBITED',
 '67124.jpg': 'SPEED_LIMIT_30',
 '67120.jpg': 'SPEED_LIMIT_30',
 '67088.jpg': 'SPEED_LIMIT_50',
 '67092.jpg': 'SPEED_LIMIT_60',
 '67128.jpg': 'SPEED_LIMIT_50',
 '63048.jpg': 'SPEED_LIMIT_15',
 '00016.jpg': 'ALL_MOTOR_VEHICLE_PROHI